# Калибровка

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import json

## Подготовка данных

Вручную отметил точку на маркерных досках, т.к. иначе получить координаты всех точек на маркерной доске, чтобы они формировали матрицу не удавалось (пробовал это сделать с помощью выделения Blob-ов, binary threshold, Hough Circle Transform, использовать модели на нейронных сетях - U-Net). Выделить маркеры в целом получалось, однако затем всё равно требуется вручную сопоставить их с координатами в системе координат относительно маркерной доски.

Итак, координаты маркерных точек находятся в файле *calib_imgs.json*. На каждом изображении 49 точек, формирующих матрицу 7 на 7. Точки затем сопоставляются с координатами самой доски в реальном 3D пространстве *objp*.

In [ ]:
imgs_folder = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                              'test_imgs')
cam1_imgs_names = glob.glob(f"{imgs_folder}/cam1_*.jpg")
cam2_imgs_names = glob.glob(f"{imgs_folder}/cam2_*.jpg")

In [ ]:
cam1_images_dict = {}
cam2_images_dict = {}

for imname in sorted(cam1_imgs_names):
    im = cv.imread(imname, 0)
    image_key = os.path.splitext(os.path.basename(imname))[0]
    cam1_images_dict[image_key] = im

for imname in sorted(cam2_imgs_names):
    im = cv.imread(imname, 0)
    image_key = os.path.splitext(os.path.basename(imname))[0]
    cam2_images_dict[image_key] = im

In [ ]:
plt.figure(figsize=(30, 15)) 
for i, img in enumerate(cam1_images_dict.values()):
    plt.subplot(1, len(cam1_images_dict), i + 1) 
    plt.imshow(cv.cvtColor(img, cv.COLOR_GRAY2RGB))
    plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(30, 15)) 
for i, img in enumerate(cam2_images_dict.values()):
    plt.subplot(1, len(cam2_images_dict), i + 1) 
    plt.imshow(cv.cvtColor(img, cv.COLOR_GRAY2RGB))
    plt.axis('off')
plt.show()

__ВАЖНО__!  
Фотку с маленькой доской (№4) использовать не будем, так как я отмечал точки на маленькой панели, а расстояния между ними меньше чем на большой панели.  
Эти фотки можно будет использовать только тогда, когда узнаем точное соотношение между расстояниями между маркерами на маленькой и большой досках. Для этого просто надо будет изменить параметр *world_scaling* в функциях калибровки.

In [ ]:
# Delete the 4-th image from images
del cam1_images_dict['cam1_4']
del cam2_images_dict['cam2_4']

In [ ]:
# Load the markers points for these images
json_path_calib_imgs = 'calib_imgs.json'
with open(json_path_calib_imgs, 'r') as file:
    circle_centers = json.load(file)

# Convert the string coordinates ("x, y") into tuples of integers
parsed_circle_centers = {}
for image_name, coordinates in circle_centers.items():
    parsed_circle_centers[image_name] = [tuple(map(int, point.split(','))) for point in coordinates]

In [ ]:
# Delete the 4-th image circles
del parsed_circle_centers['cam1_4']
del parsed_circle_centers['cam2_4']

In [ ]:
def plot_image_with_centers(image, centers):
    image_rgb = cv.cvtColor(image, cv.COLOR_GRAY2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(image_rgb)
    centers_np = np.array(centers, dtype=np.float32)
    for center in centers_np:
        plt.scatter(center[0], center[1], color='red', s=50)  # Red dots for centers
    plt.title('Image with Circle Centers')
    plt.show()

### Пример размеченной фотки

In [ ]:
img_name = 'cam1_2'
img = cam1_images_dict[img_name]
plot_image_with_centers(img, parsed_circle_centers[img_name])

## Вычисление camera matrix

Camera matrix можно вычислить исходя из следующих параметров:
1. фокусное расстояние: 80mm
2. размер пикселя: 4.5 µm × 4.5 µm
3. разрешение изображения: 5120 × 4096 

In [ ]:
def calculate_camera_matrix(focal_length_mm, image_resolution, pixel_size_x, pixel_size_y):
    """
    Calculate the camera intrinsic matrix given focal length, image resolution, and pixel size.

    Args:
        focal_length_mm (float): Focal length of the camera in millimeters.
        image_resolution (tuple): Image resolution as (width, height) in pixels.
        pixel_size_x (float): Pixel size along the x-axis (in mm).
        pixel_size_y (float): Pixel size along the y-axis (in mm).

    Returns:
        np.array: 3x3 camera intrinsic matrix.
    """
    width, height = image_resolution

    f_x = focal_length_mm / pixel_size_x
    f_y = focal_length_mm / pixel_size_y

    # Assume optical center (c_x, c_y) is at the center of the image
    c_x = width / 2.0
    c_y = height / 2.0

    camera_matrix = np.array([
        [f_x, 0,    c_x],
        [0,   f_y,  c_y],
        [0,   0,    1]
    ])

    return camera_matrix

In [ ]:
camera_params = {
    'focal_length_mm': 80,
    'image_resolution': (5120, 4096),
    'pixel_size_x': 4.5e-3,
    'pixel_size_y': 4.5e-3
}

In [ ]:
CM = calculate_camera_matrix(**camera_params)
CM

## Поиск вектора искажений

У нас уже есть матрица камеры и вычислим коэффициенты искажения с использованием калибровочных изображений и отмеченных вручную маркерах на них (*test_imgs/* и *calib_imgs.json*).

calibrateCamera принимает начальное предположение для матрицы камеры (mtx_initial_guess) и вычисляет только вектор искажения для этого зафиксируем флаг cv.CALIB_FIX_INTRINSIC.

In [ ]:
def calibrate_distortion_with_circles(images,
                                      img_dims,
                                      circle_centers,
                                      mtx_known,
                                      dist_initial_guess=None):
    rows = 7
    columns = 7
    world_scaling = 10.  # Real-world scaling: in mm

    # 3D world coordinates of circles
    objp = np.zeros((rows * columns, 3), np.float32)
    objp[:, :2] = np.mgrid[0:rows, 0:columns].T.reshape(-1, 2)
    objp *= world_scaling

    # Initialize arrays for image and object points
    objpoints = []  # 3D points in real world space
    imgpoints = []  # 2D points in image plane
    
    for img_name in images:
        corners = np.array(circle_centers[img_name], dtype=np.float32)
        imgpoints.append(corners)
        objpoints.append(objp)

    ret, _, dist, _, _ = cv.calibrateCamera(objpoints, imgpoints, img_dims, 
                                            mtx_known, dist_initial_guess, 
                                            flags=cv.CALIB_FIX_INTRINSIC)
    print('RMSE:', ret)
    
    return dist


In [ ]:
allcam_images_dict = {**cam1_images_dict, **cam2_images_dict}

In [ ]:
dist = calibrate_distortion_with_circles(allcam_images_dict, 
                                  camera_params['image_resolution'], 
                                  parsed_circle_centers, 
                                  CM)
dist

## Стерео-калибровка

In [ ]:
def stereo_calibrate(cam1_imgs, cam2_imgs, img_dims, circle_centers, CM, dist):
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 1000, 0.0001)
        
    rows = 7
    columns = 7
    world_scaling = 10.  # Real-world scaling: in mm

    # 3D world coordinates of circles
    objp = np.zeros((rows * columns, 3), np.float32)
    objp[:, :2] = np.mgrid[0:rows, 0:columns].T.reshape(-1, 2)
    objp *= world_scaling

    # Initialize arrays for image and object points
    objpoints = []  # 3D points in real world space
    imgpoints_cam1 = []
    imgpoints_cam2 = []

    # Process each image
    for cam1_img_name, cam2_img_name in zip(cam1_imgs, cam2_imgs):
        objpoints.append(objp)
        imgpoints_cam1.append(np.array(circle_centers[cam1_img_name], dtype=np.float32))
        imgpoints_cam2.append(np.array(circle_centers[cam2_img_name], dtype=np.float32))
        
    stereocalibration_flags = cv.CALIB_FIX_INTRINSIC
    ret, _, _, _, _, R, T, E, F = cv.stereoCalibrate(objpoints, 
                                                    imgpoints_cam1, 
                                                    imgpoints_cam2, 
                                                    CM, dist,
                                                    CM, dist, 
                                                    img_dims, 
                                                    criteria = criteria, 
                                                    flags = stereocalibration_flags)
 
    print(ret)
    return R, T, E, F

R: The relative rotation matrix between the first and second cameras. This matrix describes how the second camera is rotated relative to the first camera.

T: The translation vector between the first and second cameras. This vector describes how far and in which direction the second camera is from the first camera.

E: The essential matrix. This matrix encodes the rotation and translation between the two cameras, and it can be used to compute the epipolar geometry (constraints for stereo matching).

F: The fundamental matrix. This is another way of representing the geometric relationship between the two cameras. It defines the epipolar lines in stereo matching.

In [ ]:
R, T, E, F = stereo_calibrate(cam1_images_dict, 
                              cam2_images_dict, 
                              camera_params['image_resolution'],
                              parsed_circle_centers, 
                              CM, dist)

## Export the camera's intrinsic and extrinsic parameters

In [ ]:
# Saving in JSON

# Convert NumPy arrays to lists for JSON serialization
calibration_data_json = {
    'CM': CM.tolist(),
    'dist': dist.tolist(),
    'R': R.tolist(),
    'T': T.tolist(),
    'E': E.tolist(),
    'F': F.tolist()
}

with open('calibration_data.json', 'w') as f:
    json.dump(calibration_data_json, f, indent=4)